In [1]:
 # Set the seed value for the notebook so the results are reproducible
from numpy.random import seed
seed(42)
import tensorflow
tensorflow.keras.__version__
import tensorflow as tf


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
import sklearn.datasets

In [3]:
cc_df = pd.read_csv('Credit_Data_Original.csv', encoding = 'utf8')


FileNotFoundError: [Errno 2] File b'Credit_Data_Original.csv' does not exist: b'Credit_Data_Original.csv'

In [ ]:
cc_df.dropna()

In [ ]:
cc_df.drop("OBS#", axis=1)


In [ ]:
X = cc_df.drop("DEFAULT", axis=1)
y = cc_df["DEFAULT"]
print(X.shape, y.shape)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=.25)

In [ ]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [ ]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [ ]:
import h5py
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=31))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [ ]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=50,
    shuffle=True,
    verbose=2
)

In [ ]:
print(f"Normal Neural Network ")
print('-----------------------')
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)


In [ ]:
# SAFE MODEL!!!
 
model.save("DEFAULT_model_trained.h5")

In [ ]:
# RELOAD!!!!! MODEL!!! YAYYYYY!!
from tensorflow.keras.models import load_model

default_model = load_model("DEFAULT_model_trained.h5")

In [ ]:
print(f"Better Check that Normal Neural Network!! Twice! ")
print('-------------------------------------------------')
model_loss, model_accuracy = default_model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)

In [ ]:
# Predicting Default OUTCOMES
encoded_predictions = default_model.predict_classes(X_test_scaled[:])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [ ]:
print(f"DEFAULT OUTCOME KEY: (DEFAULT YES: 1 , DEFAULt NO: 0 )")
print(f"------------------------------------------------------")
print(f"Predicted Outcome: {prediction_labels}")
print(f"Actual Outcome: {list(y_test[:])}")

In [ ]:
#SAVE VARIABLES TO PLOT 

X_predict = prediction_labels
print(X_predict)
Actual_Outcome= list(y_test[:])
print(Actual_Outcome)

In [ ]:
approval_df =pd.DataFrame({"Prediction Acct Default": X_predict, "Actual Acct Default": Actual_Outcome}).reset_index(drop=True)
approval_df

In [ ]:
approval_df["Approval_Status"] =""
approval_df

In [ ]:
def Approval_Status(c):
    if c['Prediction Acct Default'] & c['Actual Acct Default']== 1:
        return 'Denied'
    elif c['Prediction Acct Default'] | c['Actual Acct Default']== 0:
        return 'Approved'
    else:
        return 'Pending Further Analysis'

In [ ]:
approval_df["Approval_Status"] =approval_df.apply(Approval_Status,axis=1)
approval_df

In [ ]:
statusValueCount= approval_df["Approval_Status"].value_counts()
statusValueCount

In [ ]:
totalStatusOutcome = len(approval_df["Approval_Status"])
totalStatusOutcome

In [ ]:
statusOutcomeRatio = statusValueCount/ totalStatusOutcome
statusOutcomeRatio 

In [ ]:
statusRatio_df =pd.DataFrame({"Total Predictive Outcomes":statusValueCount,"Percentage Predictive Outcomes":statusOutcomeRatio})
statusRatio_df

In [ ]:
statusLabel =["Approved","Pending Further Analysis","Denied"]

totals= statusRatio_df['Total Predictive Outcomes']

barList = plt.bar(statusLabel,totals)
#colors 
barList[0].set_color('aqua')
barList[1].set_color('orange')
barList[2].set_color('yellow')
# Orient widths. Add labels, tick marks, etc. 
plt.yticks(np.arange(0,200,50))
plt.title("Total Predictive Outcomes")
plt.xlabel("Approval Status")
plt.ylabel("Total Result (Based on 250 Data Input)")
plt.grid()

# Save the Figure
plt.savefig("PNG/PNG_1st_SimpleBar.png")

# Show the Figure
plt.show()

In [ ]:
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from numpy.random import rand

colors = cm.hsv(totals / float(max(totals)))
plot = plt.scatter(totals, totals, c = totals, cmap = 'hsv')
plt.clf()
plt.colorbar(plot)
plt.bar(range(len(totals)), totals, color = colors)



In [ ]:
# two y-axes, using twinx()

# statusRatio_df[["Total Predictive Outcomes","Percentage Predictive Outcomes"]].plot(kind='bar', use_index=True)
# statusRatio_df[["Total Predictive Outcomes","Percentage Predictive Outcomes"]].plot(kind='line', use_index=True)
import matplotlib.pyplot as plt
fig = plt.figure()
ax = statusRatio_df[["Total Predictive Outcomes"]].plot(kind='bar',color= "lightblue", use_index=True)
ax2 = ax.twinx()
ax2.plot(ax.get_xticks(),
         statusRatio_df[["Percentage Predictive Outcomes"]].values,
         linestyle='-',
         marker='o', linewidth=2.0,color= "gray")
# Orient widths. Add labels, tick marks, etc. 

plt.title("Total Predictive Outcomes(Bar)/Percentage Predictive Outcomes(Line)")
plt.xlabel("Approval Status")
plt.ylabel("% Predictive on linePlot")
plt.xticks(rotation=70)
# plt.savefig("PNG_barChart_lineChart_2axesForY.png")

In [ ]:
statusOutcomeRatio_asObject = ((statusValueCount/ totalStatusOutcome)*100).round(1).astype(str) + '%' 
statusOutcomeRatio_asObject

In [ ]:
statusRatio_prettyDF =pd.DataFrame({"Total Predictive Outcomes":statusValueCount,"Percentage Predictive Outcomes":statusOutcomeRatio_asObject})


print('                 AGGREGATED PREDICTIVE OUTCOMES DataFrame')
print('                _________________________________________')
statusRatio_prettyDF